In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import relu,linear
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam

import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)


tf.keras.backend.set_floatx('float64')


tf.autograph.set_verbosity(0)

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Breast_cancer_dataset.csv to Breast_cancer_dataset.csv


In [ ]:
import pandas as pd
import io

df = pd.read_csv(io.BytesIO(uploaded['Breast_cancer_dataset.csv']))

In [ ]:
print("Dataset shape:", df.shape)
print("Column names:", df.columns.tolist())
print("\nFirst 5 rows:")
print(df.head())
print("\nData types:")
print(df.dtypes)

Dataset shape: (569, 33)
Column names: ['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean', 'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se', 'fractal_dimension_se', 'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst', 'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32']

First 5 rows:
         id diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
0    842302         M        17.99         10.38          122.80     1001.0   
1    842517         M        20.57         17.77          132.90     1326.0   
2  84300903         M        19.69         21.25          130.00     1203.0   
3  84348301         M        11.42         20.38  

In [ ]:
target_column = 'diagnosis'
id_column = 'id'
another = 'Unnamed: 32'
X = df.drop(columns=[id_column, target_column, another])
y = df[target_column]

In [ ]:
# Verify what you have
print("=== Data Verification ===")
print("Dataset shape:", df.shape)
print("Target column:", target_column)
print("ID column:", id_column)

print("\nFeatures (X):")
print("  Shape:", X.shape)
print("  Columns:", list(X.columns))
print("  Data type:", X.dtypes.iloc[0])  # Check first column's dtype

print("\nTarget (y):")
print("  Shape:", y.shape)
print("  Data type:", y.dtype)
print("  Unique values:", y.unique())
print("  Value counts:")
print(y.value_counts())

print("\nFirst few rows:")
print("X (features):")
print(X.head(2))
print("\ny (target):")
print(y.head(10))

=== Data Verification ===
Dataset shape: (569, 33)
Target column: diagnosis
ID column: id

Features (X):
  Shape: (569, 30)
  Columns: ['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean', 'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se', 'fractal_dimension_se', 'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst', 'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst']
  Data type: float64

Target (y):
  Shape: (569,)
  Data type: object
  Unique values: ['M' 'B']
  Value counts:
diagnosis
B    357
M    212
Name: count, dtype: int64

First few rows:
X (features):
   radius_mean  texture_mean  perimeter_mean  area_mean  smoothness_mean  \
0        17.99         10.38           122.8 

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

# 1. Encode the target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print("Label encoding:")
print("'B' ->", label_encoder.transform(['B'])[0])
print("'M' ->", label_encoder.transform(['M'])[0])

# 2. Ensure features are numeric
X_numeric = X.astype(np.float32)

Label encoding:
'B' -> 0
'M' -> 1


In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
# If your labels are 'M'/'B' strings, encode them
if y.dtype == 'object':  # String labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)  # M=1, B=0 (or vice versa)
else:  # Already numeric
    y_encoded = y

print("Label encoding:")
if y.dtype == 'object':
    print("M =", label_encoder.transform(['M'])[0])
    print("B =", label_encoder.transform(['B'])[0])
else:
    print("0 = Malignant, 1 = Benign")

Label encoding:
M = 1
B = 0


In [ ]:
print("Dataset shape:", df.shape)
print("Column names:", df.columns.tolist())
print("\nFirst 5 rows:")
print(df.head())
print("\nData types:")
print(df.dtypes)

Dataset shape: (569, 33)
Column names: ['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean', 'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se', 'fractal_dimension_se', 'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst', 'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32']

First 5 rows:
         id diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
0    842302         M        17.99         10.38          122.80     1001.0   
1    842517         M        20.57         17.77          132.90     1326.0   
2  84300903         M        19.69         21.25          130.00     1203.0   
3  84348301         M        11.42         20.38  

In [ ]:
# 7. Apply the train-test split
X_train, X_, y_train, y_ = train_test_split(X, y_encoded, test_size=0.4, random_state=1)
X_cv, X_test, y_cv, y_test = train_test_split(X_,y_,test_size=0.50, random_state=1)

# 8. Print the shapes
print("X_train.shape", X_train.shape, "y_train.shape", y_train.shape)
print("X_test.shape", X_test.shape, "y_test.shape", y_test.shape)
print("X_cv.shape", X_cv.shape, "y_cv.shape", y_cv.shape)

X_train.shape (341, 30) y_train.shape (341,)
X_test.shape (114, 30) y_test.shape (114,)
X_cv.shape (114, 30) y_cv.shape (114,)


In [ ]:
def eval_binary_error(y_true, y_pred):
    """Calculate binary classification error rate"""
    m = len(y_true)
    incorrect = [1 if y_i != yhat_i else 0 for y_i, yhat_i in zip(y_true, y_pred)]
    error_rate = np.sum(incorrect) / m
    return error_rate


In [ ]:
tf.random.set_seed(1234)
model_simple = Sequential([
    Dense(units=16, activation='relu', name='Layer_1', input_shape=(X_train.shape[1],)),
    Dense(units=8, activation='relu', name='Layer_2'),
    Dense(units=1, activation='sigmoid', name='Output'),  # Sigmoid for binary classification
], name="Simple_Tumor_Classifier")


model_simple.compile(
    loss=BinaryCrossentropy(),  # Binary crossentropy for binary classification
    optimizer=Adam(0.001),
    metrics=['accuracy']
)

print("Simple Model Summary:")
model_simple.summary()

Simple Model Summary:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "Simple_Tumor_Classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Layer_1 (Dense)                 │ (None, 16)             │           496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Layer_2 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output (Dense)                  │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 641 (5.01 KB)

 Trainable params: 641 (5.01 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Verify your labels are correct
print("y_cv dtype:", y_cv.dtype)
print("y_cv values:", np.unique(y_cv))

# Labels should be 0 and 1, not something else
if set(np.unique(y_cv)) != {0, 1}:
    print("❌ Labels are not 0/1 - this can cause NaN loss!")
else:
    print("✓ Labels are correctly encoded as 0/1")

y_cv dtype: int64
y_cv values: [0 1]
✓ Labels are correctly encoded as 0/1


In [ ]:
history_simple = model_simple.fit(X_train, y_train,
                                 epochs=500)

Epoch 1/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9153 - loss: 0.2895 
Epoch 2/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9153 - loss: 0.2901 
Epoch 3/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9153 - loss: 0.2902  
Epoch 4/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9153 - loss: 0.2899 
Epoch 5/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9161 - loss: 0.2891 
Epoch 6/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9161 - loss: 0.2880 
Epoch 7/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9151 - loss: 0.2866 
Epoch 8/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9151 - loss: 0.2851 
Epoch 9/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9151 - loss: 0.2836 
Epoch 10/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9151 - loss: 0.2820 
Epoch 11/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9151 - loss: 0.2806 
Epoch 12/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step

In [ ]:
def eval_cat_err(y, yhat):
    """
    Calculate the categorization error
    Args:
      y    : (ndarray  Shape (m,) or (m,1))  target value of each example
      yhat : (ndarray  Shape (m,) or (m,1))  predicted value of each example
    Returns:|
      cerr: (scalar)
    """
    m = len(y)
    incorrect = 0
    #for i in range(m):
    ### START CODE HERE ###
    incorrect_ttl = [1 if y_i!=yhat_i else 0 for y_i, yhat_i in zip(y, yhat)]
    cerr = np.sum(incorrect_ttl)/m
    ### END CODE HERE ###

    return(cerr)

In [ ]:
# For binary classification (your tumor problem):
model_predict = lambda Xl: (model_simple.predict(Xl) > 0.5).astype(int).flatten()

# Since you have binary output with sigmoid, no need for softmax/argmax


In [ ]:
training_cerr_complex = eval_cat_err(y_train, model_predict(X_train))
cv_cerr_complex = eval_cat_err(y_cv, model_predict(X_cv))
print(f"categorization error, training, complex model: {training_cerr_complex:0.3f}")
print(f"categorization error, cv,       complex model: {cv_cerr_complex:0.3f}")

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
categorization error, training, complex model: 0.056
categorization error, cv,       complex model: 0.096


In [ ]:
tf.random.set_seed(1234)
model_regularized = Sequential([
    Dense(units=16, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.1), name='Layer_1', input_shape=(X_train.shape[1],)),
    Dense(units=8, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.1), name='Layer_2'),
    Dense(units=1, activation='sigmoid', name='Output'),  # Sigmoid for binary classification
], name="Simple_Tumor_Classifier")


model_regularized.compile(
    loss=BinaryCrossentropy(),  # Binary crossentropy for binary classification
    optimizer=Adam(0.001),
    metrics=['accuracy']
)

print("Simple Model Summary:")
model_simple.summary()

Simple Model Summary:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "Simple_Tumor_Classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Layer_1 (Dense)                 │ (None, 16)             │           496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Layer_2 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output (Dense)                  │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,925 (15.04 KB)

 Trainable params: 641 (5.01 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,284 (10.03 KB)

In [ ]:
history_simple_2 = model_regularized.fit(X_train, y_train,
                                 epochs=500)

Epoch 1/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3103 - loss: 48.1565
Epoch 2/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1576 - loss: 36.9816 
Epoch 3/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2018 - loss: 32.9696 
Epoch 4/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2190 - loss: 29.0335 
Epoch 5/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2177 - loss: 22.9125  
Epoch 6/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2299 - loss: 12.9578 
Epoch 7/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3628 - loss: 5.4893 
Epoch 8/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8017 - loss: 2.6686 
Epoch 9/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8581 - loss: 2.4531 
Epoch 10/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8732 - loss: 2.3228 
Epoch 11/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8838 - loss: 2.1388 
Epoch 12/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms

In [ ]:
model_predict_2 = lambda Xl: (model_regularized.predict(Xl) > 0.5).astype(int).flatten()

training_cerr_complex = eval_cat_err(y_train, model_predict_2(X_train))
cv_cerr_complex = eval_cat_err(y_cv, model_predict(X_cv))
print(f"categorization error, training, complex model: {training_cerr_complex:0.3f}")
print(f"categorization error, cv,       complex model: {cv_cerr_complex:0.3f}")

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
categorization error, training, complex model: 0.070
categorization error, cv,       complex model: 0.096


In [ ]:
tf.random.set_seed(1234)
lambdas = [0.0, 0.001, 0.01, 0.05, 0.1, 0.2, 0.3]
models=[None] * len(lambdas)
for i in range(len(lambdas)):
    lambda_ = lambdas[i]
    models[i] =  Sequential(
        [
            Dense(units=16, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.1), name='Layer_1', input_shape=(X_train.shape[1],)),
            Dense(units=8, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.1), name='Layer_2'),
            Dense(units = 1, activation = 'sigmoid', name = 'output')
        ]
    )

    models[i].compile(
      loss=BinaryCrossentropy(),  # Binary crossentropy for binary classification
      optimizer=Adam(0.001),
      metrics=['accuracy']
    )


    models[i].fit(
        X_train,y_train,
        epochs=1000
    )
    print(f"Finished lambda = {lambda_}")


Epoch 1/1000


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.4750 - loss: 5.0483
Epoch 2/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7221 - loss: 3.8639 
Epoch 3/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7672 - loss: 3.5205 
Epoch 4/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8045 - loss: 3.2545 
Epoch 5/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8045 - loss: 3.0502 
Epoch 6/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8130 - loss: 2.7890 
Epoch 7/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8517 - loss: 2.5706 
Epoch 8/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8140 - loss: 2.3993 
Epoch 9/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8675 - loss: 2.2330 
Epoch 10/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8368 - loss: 2.1013 
Epoch 11/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8719 - loss: 1.9774 
Epoch 12/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - 

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6049 - loss: 215.9623
Epoch 2/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6049 - loss: 184.6868 
Epoch 3/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6049 - loss: 158.0996 
Epoch 4/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6049 - loss: 135.2536 
Epoch 5/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6049 - loss: 115.4453 
Epoch 6/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6049 - loss: 98.2287 
Epoch 7/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6049 - loss: 82.7678 
Epoch 8/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6049 - loss: 68.3815 
Epoch 9/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6049 - loss: 54.6835 
Epoch 10/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6049 - loss: 41.4376 
Epoch 11/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6049 - loss: 28.3457 
Epoch 12/1000
11/11 ━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2149 - loss: 20.6926
Epoch 2/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1337 - loss: 18.9161 
Epoch 3/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1852 - loss: 14.5785 
Epoch 4/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4937 - loss: 4.5471 
Epoch 5/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8176 - loss: 3.0719 
Epoch 6/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8675 - loss: 2.8050 
Epoch 7/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8831 - loss: 2.5876 
Epoch 8/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8971 - loss: 2.4349 
Epoch 9/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8946 - loss: 2.3123 
Epoch 10/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8935 - loss: 2.2050 
Epoch 11/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8905 - loss: 2.1068 
Epoch 12/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step

KeyboardInterrupt: 